In [12]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

In [13]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/goutham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/goutham/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Test.xlsx')

In [15]:
train.drop_duplicates(inplace = True)

In [16]:
def punc_remover(raw_text):
    str = string.punctuation
    final_text = "".join([w for w in raw_text if w not in str])
    return final_text

In [17]:
def stopword_remover(raw_text):
    final_words = raw_text.split()
    return " ".join([w for w in final_words if w not in stopwords.words('english')])

In [18]:
def lemmatization(raw_text):
    lemm = nltk.stem.WordNetLemmatizer()
    return " ".join([lemm.lemmatize(w,'v') for w in raw_text.split()])

In [19]:
def preprocess_text(raw_text):
    clean_text = stopword_remover(punc_remover(raw_text))
    return lemmatization(clean_text)

In [20]:
train['FINAL_STORY'] = train['STORY'].apply(preprocess_text)

In [21]:
train['FINAL_STORY']

0       But painful huge reversal fee income unheard a...
1       How formidable opposition alliance among Congr...
2       Most Asian currencies trade lower today South ...
3       If want answer question click ‘Answer’ After c...
4       In global market gold price edge today disappo...
                              ...                        
7623    Karnataka Congress bastion also give BJP first...
7624    The film also feature Janhvi Kapoor revolve ar...
7625    The database create bring together criminal re...
7626    The state uneasy relationship mainland since d...
7627    Virus star Kunchacko Boban Tovino Thomas Indra...
Name: FINAL_STORY, Length: 7551, dtype: object

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
bow_dict = CountVectorizer().fit(train['FINAL_STORY'])
final_bow = bow_dict.transform(train['FINAL_STORY'])
final_bow.shape



In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
pipe = Pipeline([('Bow', CountVectorizer()),('TfIdf', TfidfTransformer()),('Classifier',MultinomialNB())])

In [25]:
pipe.fit(train['FINAL_STORY'],train['SECTION'])

Pipeline(memory=None,
         steps=[('Bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('TfIdf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('Classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
test['FINAL_STORY'] = test['STORY'].apply(preprocess_text)

In [27]:
pred = pipe.predict(test['FINAL_STORY'])